Conduct RFM (recency, frequency, monetary value) analysis on political donors in the UK

In [18]:
#imports
import pandas as pd
import datetime as dt 
import numpy as np

# DATA PROCESSING

In [19]:
#read data
df = pd.read_csv('data/Donations accepted by political parties.csv')
df.shape

/var/folders/9t/ywq9gg3s4zbgn2zw5tfskbv00000gn/T/ipykernel_23597/3107819911.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/Donations accepted by political parties.csv')


(65278, 29)

In [20]:

#drop all nans from the columns that we will be using
df = df.dropna(subset=['Value', 'DonorId', 'ReceivedDate'])
df.shape

(62492, 29)

In [21]:
#change received data from string to datetime
df['ReceivedDate'] = pd.to_datetime(df['ReceivedDate'], format='%d/%m/%Y')
df['Value'] = df['Value'].replace('[\£,]', '', regex=True).astype(float)



# RFM ANALYSIS 

In [22]:
#Compute recency values
recency_df = df.groupby(by='DonorId', as_index=False)['ReceivedDate'].max()
recency_df.columns = ['DonorId', 'LastPurchaseDate']
recent_date = recency_df['LastPurchaseDate'].max()
recency_df['Recency'] = recency_df['LastPurchaseDate'].apply(
    lambda x: (recent_date - x).days)
recency_df.head

<bound method NDFrame.head of        DonorId LastPurchaseDate  Recency
0          1.0       2001-05-14     6685
1          2.0       2003-08-11     5866
2          3.0       2015-02-18     1657
3          4.0       2010-12-31     3167
4          5.0       2009-12-15     3548
...        ...              ...      ...
42183  87473.0       2019-08-05       28
42184  87478.0       2018-11-01      305
42185  87479.0       2009-12-29     3534
42186  87480.0       2017-03-13      903
42187  87482.0       2014-02-06     2034

[42188 rows x 3 columns]>

In [23]:
#Compute Frequency 
frequency_df = df.drop_duplicates().groupby(
    by=['DonorId'], as_index=False)['ReceivedDate'].count()
frequency_df.columns = ['CustomerName', 'Frequency']
frequency_df.head

<bound method NDFrame.head of        CustomerName  Frequency
0               1.0          2
1               2.0          2
2               3.0          3
3               4.0         10
4               5.0          3
...             ...        ...
42183       87473.0          1
42184       87478.0          2
42185       87479.0          1
42186       87480.0          1
42187       87482.0          1

[42188 rows x 2 columns]>

In [24]:
#Compute Monetary Value
monetary_df = df.groupby(by='DonorId', as_index=False)['Value'].sum()
monetary_df.columns = ['CustomerName', 'Monetary']
monetary_df.head

<bound method NDFrame.head of        CustomerName   Monetary
0               1.0   53000.00
1               2.0  118680.84
2               3.0   26625.00
3               4.0   59495.89
4               5.0    7700.00
...             ...        ...
42183       87473.0    4000.00
42184       87478.0    3529.41
42185       87479.0   11520.00
42186       87480.0  157350.07
42187       87482.0   50000.00

[42188 rows x 2 columns]>

In [ ]:
#merge R, F & M values into one df
rfm = pd.merge()